# S1: MCMTpy Notebook

In this notebook, we will show some of the key steps in **MCMTpy** for **Focal Mechanism Inversion**, including **DC** (double couple) and **MT** (moment tensor) inversion (not in now). 

The steps to do inversion process are:    
* Compute Green Function Database
* Synthesize the Test Data
* Prepare Data for MCMTpy
* Inversion of DC
* Inversion of MT
* Result Visualization


More details on the descriptions of data processing, parameters can be found in the online [documentations](https://github.com/OUCyf) and our paper.

`MCMTpy: A Python Package for Simultaneous Inversion of Source Location, Focal Mechanism, and Rupture Directivity. In prep for Seismological Research Letter.`



Fu Yin

School of Earth and Space Sciences

University of Science and Technology of China

No.96, JinZhai Road Baohe District, Hefei, Anhui, 230026, P.R.China.

June 2021

## Building env for MCMTpy

Before running this notebook, make sure that you have created and activated the conda env made for MCMTpy. If not, you can create one using command lines below (note that jupyter is installed with the command lines here in order to run this notebook). 

```bash
$ conda create -n MCMTpy  python=3.8 numpy=1.16 matplotlib=3.1.1 mpi4py obspy pyasdf json5 tqdm
$ conda activate MCMTpy
$ pip install pyfk
$ pip install MCMTpy
```

Then you need to activate this notebook with the newly built MCMTpy env by invoking the jupyter with the following command line.

```bash
$ jupyter notebook
```

Now we can begin to load the modules needed for this practise. 

- First, let's run the cell below to import packages and the last cell with [helper functions](#helper).
- Back from Helper Function
<a id='helper_back'></a>

In [94]:
import os
import sys
import glob
import math
import obspy
import json5
import pyasdf
import shutil
# import pygmt
import numpy as np
import matplotlib.pyplot as plt
from obspy import Stream
from obspy.taup import TauPyModel
from IPython.display import Image
from MCMTpy.utils.asdf_function import get_StationXML,Add_waveforms_head_info
from MCMTpy.utils.asdf_function import Add_waveforms,Add_stationxml,get_QuakeML,Add_quakeml


## 1. Setup Green Function Database

- The first step of **MCMTpy** is to build Green's function database. All parameters are written into a JSON file named **build_GFs.json**. Some brief descriptions of the parameters are included here following the definination, but note that more details on this can be found in documentations. 

- We do not recommend using relative paths because of the possibility of errors. Absolute paths are preferred.

In [ ]:
# The root directory of the project
example_path = '/Users/yf/3.Project/8.MCMTpy/MCMTpy-master/data/example_yunnan'


#------------------------------------------------------------#
# we expect no parameters need to be changed below
#------------------------------------------------------------#
# 1. get notebook path
notebook_path = sys.path[0]
os.chdir(notebook_path)

# 2. change path to Green_Funcs
os.chdir('../Green_Funcs')
print(os.listdir())

# 3. show build_GFs.json 
filename = 'build_GFs.json'
with open(filename, 'r',encoding='utf-8') as f1:
    gfs_json=json5.load(f1)
f1.close()

# 4. change parameters with absolute path
gfs_json['Source_Station_info'] = os.path.join(example_path,"Green_Funcs/Source_Station_info.txt")
gfs_json["DATADIR"] = os.path.join(example_path,"Green_Funcs/GFs")
gfs_json["DATADIR_splits"] = os.path.join(example_path,"Green_Funcs/GFs/GFs_splits")
gfs_json["Velocity_model"] =  os.path.join(example_path,"v_model/v_model_yunnan.txt")        
gfs_json_new = os.path.join(example_path,'Green_Funcs/build_GFs_new.json')

with open(gfs_json_new,'w') as f2:
    json5.dump(gfs_json, f2, indent=2)
f2.close()

gfs_json

**NOTE**: if you want to make any changes of parameters to the json-file, please open it with a text editor and make the changes manually. It is strongly recommended to use a separate directory apart from the $notebook_path as the **GF databases can become very large**, depending on the different problem! For real examples, the GF databases may require up to several GBs of free disc space. For quick calculations, we use **'Database_mode': False**. Now we will setup GFs datebase!

In [7]:
# run MCMTpy in shell
# NOTE: too long running time，please comment out!
# !mpirun -n 4 MCMTpy build_GFs pyfk -c build_GFs_new.json  > gfs.log

os.chdir(notebook_path)
os.chdir('../Green_Funcs/GFs')
print(os.listdir())

['.DS_Store', 'GFs_splits', 'prepro_para_info.txt', 'write_Source_Station_info_MPI.txt']


- MCMTpy will generates 2 logging files when it builds GFs datebase, that is **prepro_para_info.txt** and **write_Source_Station_info_MPI**. They record all the parameter information of GFs datebase.

In [ ]:
# MCMTpy generate 2 logging file

os.chdir(notebook_path)
os.chdir('../Green_Funcs/GFs')
print('************************************')
print('prepro_para_info.txt:\n\n')
!cat prepro_para_info.txt

print('\n\n\n************************************')
print('write_Source_Station_info_MPI.txt:\n\n\n')
# !cat write_Source_Station_info_MPI.txt

## 2. Syn the Test Data

- The synthesized test data can help us check whether the Green's function database is correctly calculated.

In [ ]:
# change path to Green_Funcs
os.chdir(notebook_path)
os.chdir('../syn')
print(os.listdir())


# show build_GFs.json 
filename = 'syn.json'
with open(filename, 'r',encoding='utf-8') as f1:
    syn_json=json5.load(f1)


# change parameters with absolute path
syn_json["GFs_json_file"] = os.path.join(example_path,"Green_Funcs/build_GFs_new.json")
syn_json["Stf_file"] = os.path.join(example_path,"syn/Stf_file/Stf_file.sac")
syn_json["Output_path"] = os.path.join(example_path,"syn/Synthetic")       
syn_json_new = os.path.join(example_path,'syn/syn_new.json')

with open(syn_json_new,'w') as f2:
    json5.dump(syn_json, f2, indent=2)
f2.close()

syn_json

- Now we will syn test data in datebase:

In [81]:
# run MCMTpy in shell

os.chdir(notebook_path)
os.chdir('../syn')

shutil.rmtree('./Synthetic')
!MCMTpy syn pyfk  -c ./syn_new.json  > syn.log

- Read data:

In [46]:
## change the path
h5_path = os.path.join(example_path,'syn/Synthetic/SYN_test.h5')
source_name = 'source_syn'


### read h5 data --> Sta_data (dict)
ds_raw=pyasdf.ASDFDataSet(h5_path,mpi=False,mode='r')                           
Station_list = ds_raw.waveforms.list()
Station_num = len(Station_list)

Sta_data={}                        
for i in range(0,Station_num,1):
    info={}
    tags = ds_raw.waveforms[Station_list[i]].get_waveform_tags()
    if source_name in tags:                                           
        raw_sac = ds_raw.waveforms[Station_list[i]][source_name]
        tp = float( ds_raw.waveforms[Station_list[i]][source_name][0].stats.asdf['labels'][1] ) 
        ts = float( ds_raw.waveforms[Station_list[i]][source_name][0].stats.asdf['labels'][3] ) 
        info.update({ "tp": tp})  
        info.update({ "ts": ts})
        info.update({ "data": raw_sac})
        Sta_data.update({ Station_list[i]: info})

# Sta_data

- View syn waveform:

In [ ]:
# plot data
ax = Sta_data['YN.YUM']['data'].plot()
Sta_data['YN.YUM']['data'][0].stats


## 3. Prepare data for MCMTpy

- View the station distribution：

In [34]:
# data path
allfiles_path = os.path.join(example_path,'YN.202105212148_Inv/YN.202105212148_raw/*.SAC')  

# stations with amplitude exceeds the range
limiting_sta = ['XG.CHT','XG.HDQ','XG.XBT','XG.ZYT','YN.TUS','YN.YUX']         

# 1. read raw data
data = read_data(allfiles_path)

# 2. pygmt plot
# fig = plot_station(data,limiting_sta)
# fig.show()


- Ray tracing:

In [ ]:
# 3. taup ray tracing
model_path = os.path.join(example_path,"v_model/v_model.npz")

# model can also be "iasp91" or "prem"
model = TauPyModel(model=model_path)
for i in range(0,len(data),1):
    depth = data[i].stats.sac['evdp']
    distance = data[i].stats.sac['dist']
    ray_p,tp,angle_p,ray_s,ts,angle_s = get_taup_tp_ts(model,depth,distance,degree=False)
    data[i].stats.sac["t1"]=tp                  
    data[i].stats.sac["t2"]=ts

    
# 4. plot raypath of the last station
path_p = model.get_ray_paths(source_depth_in_km=depth,
                             distance_in_degree=distance/111.19,
                             phase_list=["p", "P","s", "S"])
path_p[0].path.dtype
ax = path_p.plot_rays(plot_type="cartesian",legend=True)                    # 'spherical'  'cartesian'


- Before data preprocessing:

In [ ]:
# last station waveform
ax = data[-3:].plot()

- Data preprocessing：

In [37]:
freqmin       = 0.005              # pre filtering frequency bandwidth (hz)
freqmax       = 2                  # note this cannot exceed Nquist frequency (hz)
samp_freq     = 5                  # targeted sampling rate (hz)
p_n0          = 50                 # number of sampling points before P wave arrives
npts          = 2048               # data length


# 5. 数据预处理
preprocess(data,freqmin,freqmax,samp_freq,p_n0,npts)

- After data preprocessing：

In [ ]:
# last station waveform
ax = data[-3:].plot()

- Output ASDF and SAC data

In [85]:
# output data/file path
Output_path   = os.path.join(example_path,"YN.202105212148_Inv/YN.202105212148_MCMTpy")                                  # 输出文件存放的位置
source_name   = "source_enz"                                                    # asdf‘s source_tag
ASDF_filename = "YN.202105212148"                                               # asdf filename

# 6. output ASDF and SAC data
shutil.rmtree(Output_path)
os.mkdir(Output_path)
write_ASDF(data,Output_path,source_name,ASDF_filename)


## 4. Inv of DC

- Change inversion parameters:

In [ ]:
# change path to Green_Funcs
os.chdir(notebook_path)

# plot alpha
N=5e3
a=1000                  # 0.2*N
b=100                   # 0.04*N
alpha_max=100
alpha_min=0

tt=np.linspace(0, round(N), num=round(N))
yy=np.zeros(len(tt))
for i in range(0,len(tt)):
    yy[i]=(alpha_max-alpha_min)*(a+math.exp((tt[0]-a)/b))/(a+math.exp((tt[i]-a)/b))+alpha_min
    
fig, axs = plt.subplots(1,1)
axs.plot(tt,yy, linestyle='-', color='red', lw=3, alpha=0.6)
axs.set_xlabel("Sample Number",fontsize=17)
axs.set_ylabel("Alpha Value",fontsize=17)

# save figure
figurename=os.path.join('./S1_figure/Alpha.png')
plt.savefig(figurename,dpi=800, format="png")

In [105]:
# change path to Green_Funcs
os.chdir(notebook_path)
os.chdir('../YN.202105212148_Inv/dc_inv')
print(os.listdir())


# show build_GFs.json 
filename = 'sample_dc.json'
with open(filename, 'r',encoding='utf-8') as f1:
    sample_dc_json=json5.load(f1)


# change parameters with absolute path
sample_dc_json["GFs_json_file"] = os.path.join(example_path,"Green_Funcs/build_GFs_new.json")
sample_dc_json["GFs_file"] =os.path.join(example_path,"Green_Funcs/GFs/GFs_splits" )          
sample_dc_json["Raw_data_file"] = os.path.join(example_path,"YN.202105212148_Inv/YN.202105212148_MCMTpy/YN.202105212148.h5")              
sample_dc_json["Output_path"] = os.path.join(example_path,"YN.202105212148_Inv/dc_inv/Output_YN.202105212148_dc")         
sample_dc_json["Raw_data_inv_info"] = os.path.join(example_path,"YN.202105212148_Inv/dc_inv/Raw_data_inv_info.txt")     
sample_dc_json["Raw_data_inv_info_writed"] = os.path.join(example_path,"YN.202105212148_Inv/dc_inv/Raw_data_inv_info_writed.txt") 
sample_dc_json_new = os.path.join(example_path,'YN.202105212148_Inv/dc_inv/sample_dc_new.json')

sample_dc_json["MPI_n"] = 4                         # CPU num
sample_dc_json["Chains_n"] = 4                      # Eack MK-chains num
sample_dc_json["N"] = 1e1                           # each chain‘s search number

sample_dc_json["alpha_max"] = 100                   # The initial value of alpha
sample_dc_json["alpha_min"] = 0                     # The final value of alpha
sample_dc_json["a"] = 10                     
sample_dc_json["b"] = 10               

with open(sample_dc_json_new,'w') as f2:
    json5.dump(sample_dc_json, f2, indent=2)
f2.close()

# sample_dc_json

['plot.log', '.DS_Store', 'figure_dc', 'sample_dc_new.json', 'Output_YN.202105212148_dc', 'sample.log', 'sample_dc.json', 'plot_dc.json', 'plot_dc_new.json', 'Raw_data_inv_info.txt']


- Now run MCMTpy:

In [100]:
# run MCMTpy in shell

os.chdir(notebook_path)
os.chdir('../YN.202105212148_Inv/dc_inv')
# !MCMTpy  sample MH  -c ./sample_dc_new.json #> sample.log

## 5. Plot Results of DC

- Change plotting parameters:

In [106]:
# change path to Green_Funcs
os.chdir(notebook_path)
os.chdir('../YN.202105212148_Inv/dc_inv')
print(os.listdir())


# read plot_dc.json 
filename = 'plot_dc.json'
with open(filename, 'r',encoding='utf-8') as f1:
    plot_dc_json=json5.load(f1)


# change parameters with absolute path
plot_dc_json["plot_output_path"] = os.path.join(example_path,"YN.202105212148_Inv/dc_inv/figure_dc")                                              
plot_dc_json["Inv_json_file"] = os.path.join(example_path,"YN.202105212148_Inv/dc_inv/sample_dc_new.json") 
plot_dc_json_new = os.path.join(example_path,'YN.202105212148_Inv/dc_inv/plot_dc_new.json')

# 1.hist
plot_dc_json["plot_hist"] =True
plot_dc_json["N_start"] = 0 
plot_dc_json["N_start_accept"] = 1
plot_dc_json["num_bins"] =  50     
plot_dc_json["num_std"] =5      
plot_dc_json["labels_name"] = ['Mw','strike/°','dip/°','rake/°','z/km']

# 2. misfit
plot_dc_json["plot_misfit"] = True
plot_dc_json["MPI_n_st"] = 0
plot_dc_json["Chains_n_st"] = 0

# 3. waveform
plot_dc_json["plot_waveform"] = True
plot_dc_json["FM_best"] = [6.68,  135.1,  87.0,  -168.1,    25.58,  99.86,  5.95,  -0.57 ]     
plot_dc_json["line_n_sta"] = 3                                     # Draw the data of several stations in a row
plot_dc_json["max_p_ylim"] = 1                                     # max amp y-axis of p wave
plot_dc_json["max_s_ylim"] = 1.0
plot_dc_json["max_surf_ylim"] = 1
plot_dc_json["plot_comp"] = [[1,1,0],[0,0,0],[1,1,1]]              # What components do you want to draw? P、S、Surf's Z/R/T


with open(plot_dc_json_new,'w') as f2:
    json5.dump(plot_dc_json, f2, indent=2)
f2.close()

# plot_dc_json

['plot.log', '.DS_Store', 'figure_dc', 'sample_dc_new.json', 'Output_YN.202105212148_dc', 'sample.log', 'sample_dc.json', 'plot_dc.json', 'plot_dc_new.json', 'Raw_data_inv_info.txt']


- Now run MCMTpy:

In [72]:
# run MCMTpy in shell
os.chdir(notebook_path)
os.chdir('../YN.202105212148_Inv/dc_inv')
!MCMTpy plot pyfk -c plot_dc_new.json > plot.log

/Users/yf/opt/anaconda3/lib/python3.8/site-packages/MCMTpy-0.1.0a1-py3.8.egg/MCMTpy/plotting/pyfk_plot/plot_misfit.py:51: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/Users/yf/opt/anaconda3/lib/python3.8/site-packages/MCMTpy-0.1.0a1-py3.8.egg/MCMTpy/plotting/pyfk_plot/plot_misfit.py:58: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/Users/yf/opt/anaconda3/lib/python3.8/site-packages/MCMTpy-0.1.0a1-py3.8.egg/MCMTpy/plotting/pyfk_plot/plot_misfit.py:68: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/Users/yf/opt/anaconda3/lib/python3.8/site-packages/MCMTpy-0.1.0a1-py3.8.egg/MCMTpy/plotting/pyfk_plot/plot_misfit.py:78: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.


- Show figure of inversion:

In [ ]:
# show hist
fig_hist = os.path.join(plot_dc_json["plot_output_path"],'hist.jpg')
Image(filename = fig_hist, width=600)

In [ ]:
# show hist_accept
fig_hist_accept = os.path.join(plot_dc_json["plot_output_path"],'hist_accept.jpg')
Image(filename = fig_hist_accept, width=600)

In [ ]:
# show misfit
fig_misfit = os.path.join(plot_dc_json["plot_output_path"],'misfit.jpg')
Image(filename = fig_misfit, width=600)

In [ ]:
# show waveform
fig_waveform = os.path.join(plot_dc_json["plot_output_path"],'waveform.jpg')
Image(filename = fig_waveform, width=1000)

# annotation:
# waveform = os.path.join(plot_dc_json["plot_output_path"],'waveform.jpg')
# fig_waveform = plt.imread(waveform)
# plt.imshow(fig_waveform)

![]('./figure_dc/hist.jpg')

## The end.

We hope you enjoy it! 

Most of the core steps of MCMTpy are included here.

#  Helper function：
<a id='helper'></a>

In [84]:
#***************************************************************
#*                       -----------
#*                        Functions
#*                       -----------
#***************************************************************
# The notobook needs some functions, please run it firstly:

#----------------------------------------------------#
# 1.read raw data
def read_data(allfiles_path):

    allfiles = sorted( glob.glob(allfiles_path) )
    data_raw = Stream()
    for i in range(0,len(allfiles),1):
        try:
            tr = obspy.read(allfiles[i])
            data_raw += tr
        except Exception:
            print(allfiles[i],': no such file or obspy read error');continue
    data = data_raw.copy()
    
    return data



#----------------------------------------------------#
# 2.plot station
def plot_station(data,limiting_sta):
    lons = []; lats = []; net = []; sta = []; names = []
    unique_stations = np.unique([tr.stats.station for tr in data])
    for station in unique_stations:
        tr = data.select(station=station)[0]
        lons.append(tr.stats.sac.stlo)
        lats.append(tr.stats.sac.stla)
        net.append(tr.stats.network)
        sta.append(tr.stats.station)
        names.append(f'{tr.stats.network}.{tr.stats.station}')

    region = [np.min(lons), np.max(lons), np.min(lats), np.max(lats)]
    x_pad = (region[1] - region[0]) * 0.1
    y_pad = (region[3] - region[2]) * 0.1
    region = [region[0] - x_pad, region[1] + x_pad, region[2] - y_pad, region[3] + y_pad]

    lon_0 = np.mean(region[:2])
    lat_0 = np.mean(region[2:])
    if lat_0 > 0:
        ref_lat = 90
    else:
        ref_lat = -90
    projection = f'S{lon_0}/{ref_lat}/6i'


    fig = pygmt.Figure()

    fig.coast(region=region, projection=projection,shorelines=True,water='lightblue',land='lightgrey',
              N=[1, 2],frame='a')

    focal_mechanism = dict(strike=141, dip=68, rake=-153, magnitude=5.2) 

    fig.meca(focal_mechanism, scale="0.5c",longitude=tr.stats.sac.evlo,latitude=tr.stats.sac.evla,
             depth=12.0,G='red')

    for i in range(0,len(names),1):
        if names[i] in limiting_sta:
            color = 'red'
        else:
            color = 'blue'
        fig.plot(lons[i], lats[i],style='i0.1i',color=color,pen=True,label='Station') 

    fig.text(text=names,x=lons,y=lats,X ='0.2c',Y ='0.2c',font="5p,Helvetica-Bold,white",
             fill="orange",transparency=20) 

    return fig
    
    

#----------------------------------------------------#
# 3.get_taup_tp_ts
def get_taup_tp_ts(model,depth,distance,degree=None):
    if degree==False:
        distance = distance/111.19

    time_p = model.get_travel_times(source_depth_in_km=depth,
                                    distance_in_degree=distance,
                                    phase_list=["p", "P"])

    time_s = model.get_travel_times(source_depth_in_km=depth,
                                    distance_in_degree=distance,
                                    phase_list=["s", "S"])

    ray_p = time_p[0].ray_param
    tp = time_p[0].time
    angle_p = time_p[0].incident_angle

    ray_s = time_s[0].ray_param
    ts = time_s[0].time
    angle_s = time_s[0].incident_angle

    return ray_p,tp,angle_p,ray_s,ts,angle_s



#----------------------------------------------------#
# 4.preprocess
def preprocess(data,freqmin,freqmax,samp_freq,p_n0,npts):

    data.detrend(type='demean')
    data.detrend(type='simple')
    data.taper(max_percentage=0.05)

    data.filter('bandpass', freqmin=freqmin, freqmax=freqmax, corners=4, zerophase=True)  # bandpass
    data.resample(samp_freq,window='hanning',no_filter=True, strict_length=False)         # resample

    for i in range(0,len(data),1):
        b = data[i].stats.sac['b']
        t1 = data[i].stats.sac['t1']
        t_start = data[i].stats.starttime + (t1-b) - p_n0/samp_freq
        t_end_npts = t_start+npts/samp_freq
        t_endtime = data[i].stats.endtime
        if t_end_npts > t_endtime:
            t_end = t_endtime
        else:
            t_end = t_end_npts

        data[i].trim(t_start, t_end)

    for i in range(0,round(len(data)/3)):
        t_start_1 = data[3*i].stats.starttime
        t_start_2 = data[3*i+1].stats.starttime
        t_start_3 = data[3*i+2].stats.starttime
        t_end_1 = data[3*i].stats.endtime
        t_end_2 = data[3*i+1].stats.endtime
        t_end_3 = data[3*i+2].stats.endtime
        t_start = max(t_start_1,t_start_2,t_start_3)
        t_end = min(t_end_1,t_end_2,t_end_3)
        data[3*i:3*i+3].trim(t_start, t_end)

    # velocity(nm/s) to velocity(cm/s)
    for i in range(0,len(data),1):
        data[i].data = 1e-6*data[i].data



#----------------------------------------------------#
# 5.write_ASDF
def write_ASDF(data_enz,Output_path,source_name,ASDF_filename):
    source_lat   = data_enz[0].stats.sac.evla
    source_lon   = data_enz[0].stats.sac.evlo
    source_depth = data_enz[0].stats.sac.evdp
    source_time  = data_enz[0].stats.starttime + data_enz[0].stats.sac.b

    Output_file_path = os.path.join(Output_path, ASDF_filename+'.h5')
    catalog_create   = get_QuakeML(source_name, source_lat, source_lon, source_depth,source_time)
    Add_quakeml(Output_file_path,catalog_create)

    Station_num = round(len(data_enz)/3)
    for i in range(0,Station_num,1): 
        net_sta_name     = data_enz[3*i].stats.network+'_'+data[3*i].stats.station
        station_network  = data_enz[3*i].stats.network
        station_name     = data_enz[3*i].stats.station
        station_lat      = data_enz[3*i].stats.sac.stla
        station_lon      = data_enz[3*i].stats.sac.stlo
        station_depth    = 0
        station_distance = data_enz[3*i].stats.sac.dist
        station_az       = data_enz[3*i].stats.sac.az
        station_baz      = data[3*i].stats.sac.baz
        tp               = data_enz[3*i].stats.sac.t1
        ts               = data_enz[3*i].stats.sac.t2
        stream           = data_enz[3*i:3*i+3].copy()
        starttime        = data_enz[3*i].stats.starttime
        time_before_first_arrival = p_n0/samp_freq

        inv=get_StationXML(station_network, 
                           station_name,
                           station_lat, 
                           station_lon, 
                           station_depth, 
                           'enz')

#         print("now syn sac "+station_network+'_'+station_name+':\n')

        Add_waveforms([stream], 
                      catalog_create,
                      Output_file_path,
                      source_name,
                      tp,
                      ts,
                      station_distance,
                      station_az,
                      station_baz) 

        Add_stationxml(inv,
                       Output_file_path) 

        for j in range(0,3,1): 
            CHN = stream[j].stats.channel
            sac_filename = os.path.join(Output_path,source_name+'.'+station_network+'.'+station_name+'.'+CHN+'.sac')
            stream[j].write(sac_filename, format='SAC')

-[Turn back!](#helper_back)